In [ ]:
import sqlite3
import pandas as pd

In [ ]:
def sample_csv(input_path, output_path, sample_size=20000, random_state=42):
    
    # Load CSV
    df = pd.read_csv(input_path)
    
    # Ensure sample size doesn’t exceed total rows
    sample_size = min(sample_size, len(df))
    
    # Take random sample
    sample_df = df.sample(n=sample_size, random_state=random_state)
    
    # Save to new CSV
    sample_df.to_csv(output_path, index=False)
    
    print(f"✅ Sampled {sample_size} rows saved to '{output_path}'")
    return sample_df


In [ ]:
sampled_data = sample_csv(
    input_path="C:/Users/Sherin/Downloads/combined_with_dbl.csv",      
    output_path="phishing_sample.csv"    # new sample file name
)

In [ ]:
from urllib.parse import urlparse


In [ ]:
#create a connection for database file
conn = sqlite3.connect('phishing_dataset.db')

In [ ]:
cursor = conn.cursor()


In [ ]:
cursor.executescript('''
CREATE TABLE IF NOT EXISTS urls (
    url_id INTEGER PRIMARY KEY AUTOINCREMENT,
    url TEXT,
    domain TEXT,
    date_added TEXT,
    verified TEXT,
    url_status TEXT,
    label TEXT
);

CREATE TABLE IF NOT EXISTS sources (
    source_id INTEGER PRIMARY KEY AUTOINCREMENT,
    source_name TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS threats (
    threat_id INTEGER PRIMARY KEY AUTOINCREMENT,
    threat_type TEXT,
    tags TEXT
);

CREATE TABLE IF NOT EXISTS reports (
    report_id INTEGER PRIMARY KEY AUTOINCREMENT,
    url_id INTEGER,
    source_id INTEGER,
    threat_id INTEGER,
    reporter TEXT,
    dbl_status TEXT,
    urlhaus_link TEXT,
    target TEXT,
    FOREIGN KEY (url_id) REFERENCES urls(url_id),
    FOREIGN KEY (source_id) REFERENCES sources(source_id),
    FOREIGN KEY (threat_id) REFERENCES threats(threat_id)
);
''')

conn.commit()


In [ ]:
data = pd.read_csv("C:/Users/Sherin/Downloads/combined_with_dbl.csv")

C:\Users\Sherin\AppData\Local\Temp\ipykernel_37984\4023697865.py:1: DtypeWarning: Columns (5,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:/Users/Sherin/Downloads/combined_with_dbl.csv")


In [ ]:
data['domain'] = data['url'].apply(lambda x: urlparse(str(x)).netloc)

In [ ]:
data.tail(10)

,url,source,label,id,dateadded,verified,url_status,target,threat,tags,urlhaus_link,reporter,dbl_status,domain
92664,http://60.23.137.176:44716/bin.sh,URLhaus,phishing,3618064.0,2025-09-06 00:25:19,NaN,offline,NaN,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3618064/,geenensp,1,60.23.137.176:44716
92665,https://rka.firu6ui8.ru/7ykknjnzxj.pdf,URLhaus,phishing,3618063.0,2025-09-06 00:24:08,NaN,offline,NaN,malware_download,ClearFake,https://urlhaus.abuse.ch/url/3618063/,anonymous,1,rka.firu6ui8.ru
92666,http://119.117.247.196:44449/i,URLhaus,phishing,3618061.0,2025-09-06 00:22:20,NaN,offline,NaN,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3618061/,geenensp,1,119.117.247.196:44449
92667,http://59.47.190.103:42509/i,URLhaus,phishing,3618062.0,2025-09-06 00:22:20,NaN,offline,NaN,malware_download,"32-bit,arm,elf,mirai,Mozi",https://urlhaus.abuse.ch/url/3618062/,geenensp,1,59.47.190.103:42509
92668,https://rka.firu6ui8.ru/xm8l57ovq6.pdf,URLhaus,phishing,3618060.0,2025-09-06 00:20:10,NaN,offline,NaN,malware_download,ClearFake,https://urlhaus.abuse.ch/url/3618060/,anonymous,1,rka.firu6ui8.ru
92669,http://39.78.221.240:59379/bin.sh,URLhaus,phishing,3618059.0,2025-09-06 00:17:12,NaN,offline,NaN,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3618059/,geenensp,1,39.78.221.240:59379
92670,https://ng.firu6ui8.ru/fl8uofgq7i.pdf,URLhaus,phishing,3618058.0,2025-09-06 00:07:09,NaN,offline,NaN,malware_download,ClearFake,https://urlhaus.abuse.ch/url/3618058/,anonymous,1,ng.firu6ui8.ru
92671,http://119.179.214.17:58609/i,URLhaus,phishing,3618057.0,2025-09-06 00:05:09,NaN,offline,NaN,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3618057/,geenensp,1,119.179.214.17:58609
92672,http://59.47.190.103:42509/bin.sh,URLhaus,phishing,3618056.0,2025-09-06 00:01:20,NaN,offline,NaN,malware_download,"32-bit,arm,elf,mirai,Mozi",https://urlhaus.abuse.ch/url/3618056/,geenensp,1,59.47.190.103:42509
92673,http://222.138.176.153:36181/i,URLhaus,phishing,3618055.0,2025-09-06 00:00:18,NaN,offline,NaN,malware_download,"32-bit,elf,mips,Mozi",https://urlhaus.abuse.ch/url/3618055/,geenensp,1,222.138.176.153:36181


In [ ]:
data['source'] = data['source'].str.strip().str.lower()


In [ ]:
print(data['source'].value_counts())


source
phishtank    50346
urlhaus      42028
openphish      300
Name: count, dtype: int64


In [ ]:
sources = data["source"].dropna().unique()

for src in sources:
    cursor.execute("INSERT OR IGNORE INTO sources (source_name) VALUES (?)", (src,))
conn.commit()


In [ ]:
threats = data[["threat", "tags"]].fillna("").drop_duplicates()

for _, row in threats.iterrows():
    cursor.execute(
        "INSERT OR IGNORE INTO threats (threat_type, tags) VALUES (?, ?)",
        (row["threat"], row["tags"])
    )
conn.commit()


In [ ]:
urls = data[["url", "domain", "dateadded", "verified", "url_status", "label"]].drop_duplicates()

for _, row in urls.iterrows():
    cursor.execute('''
        INSERT OR IGNORE INTO urls (url, domain, date_added, verified, url_status, label)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        row["url"],
        row["domain"],
        row.get("dateadded", None),
        row.get("verified", None),
        row.get("url_status", None),
        row.get("label", None)
    ))
conn.commit()


In [ ]:
for _, row in data.iterrows():
    # Get URL foreign key
    cursor.execute("SELECT url_id FROM urls WHERE url = ?", (row["url"],))
    url_id = cursor.fetchone()

    # Get Source foreign key
    cursor.execute("SELECT source_id FROM sources WHERE source_name = ?", (row["source"],))
    source_id = cursor.fetchone()

    # Get Threat foreign key
    cursor.execute("SELECT threat_id FROM threats WHERE threat_type = ?", (row["threat"],))
    threat_id = cursor.fetchone()

    # Insert into reports
    cursor.execute('''
        INSERT INTO reports (url_id, source_id, threat_id, reporter, dbl_status, urlhaus_link, target)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        url_id[0] if url_id else None,
        source_id[0] if source_id else None,
        threat_id[0] if threat_id else None,
        row.get("reporter", None),
        row.get("dbl_status", None),
        row.get("urlhaus_link", None),
        row.get("target", None)
    ))

conn.commit()


In [ ]:
print("URLs:", cursor.execute("SELECT COUNT(*) FROM urls").fetchone()[0])
print("Sources:", cursor.execute("SELECT COUNT(*) FROM sources").fetchone()[0])
print("Threats:", cursor.execute("SELECT COUNT(*) FROM threats").fetchone()[0])
print("Reports:", cursor.execute("SELECT COUNT(*) FROM reports").fetchone()[0])


URLs: 92674
Sources: 3
Threats: 580
Reports: 48059
